In [3]:
import pandas as pd

In [4]:
import logging
logging.getLogger().setLevel(logging.INFO)
# DEBUG - what developers look at
# INFO - extra information on what's happening
# logging.info(f"The age ({age}) is between 24 and 34")
# WARNING - this won't stop your code from running but you're doing something slightly off
# EXCEPTION / ERROR - this will stop your code but you'll get extra info on the error

In [1]:
from src import eda

In [5]:
path2="../datasets/release_data.csv"

data_2 = pd.read_csv(path2)
df2 = data_2.copy()
df2.sample(10)

,release_id,country,year,genre,format
5602703,4912990,France,1973.0,Stage & Screen,Vinyl
13046564,11852426,UK,NaN,Reggae,Vinyl
15265927,13835008,US,1966.0,Pop,Vinyl
3169578,2853840,UK & Europe,2010.0,"Folk, World, & Country",Vinyl
9983284,8923642,NaN,2016.0,"Folk, World, & Country",Vinyl
13439722,12204448,UK & Europe,2001.0,Rock,Cassette
6388913,5627182,Europe,1993.0,Hip Hop,CD
3508753,3129121,Canada,1988.0,Rock,Vinyl
15043350,13637621,Europe,2019.0,Pop,DVD
81561,77846,Italy,1992.0,Electronic,Vinyl


In [6]:
eda.numerical_data_columns(df2)

['release_id', 'year']